In [1]:
#local
import csv
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup
model_id = "meta-llama/Llama-3.2-3B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    trust_remote_code=True
)

# Hindi prompts
prompts = [
    "भारत का प्रधानमंत्री कौन है?",
    "ताजमहल कहाँ स्थित है?",
    "पानी क्यों जरूरी होता है?",
    "भारत में कितने राज्य हैं?",
    "स्वास्थ्य ही धन क्यों है?",
    "गर्मी से कैसे बचा जा सकता है?",
    "आपका नाम क्या है?",
    "कृपया मुझे मौसम के बारे में बताइए।",
    "कंप्यूटर कैसे काम करता है?",
    "पृथ्वी सूरज के चारों ओर क्यों घूमती है?"
]

# CSV output
csv_path = "llm_streaming_llama_3B_latency.csv"
with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Prompt_ID", "Prompt", "Trial", "TTFT_ms", "TPS"])

    for idx, prompt in enumerate(prompts, 1):
        for trial in range(1, 4):
            input_ids = tokenizer.apply_chat_template(
                [
                    {"role": "system", "content": "आप एक मददगार AI सहायक हैं। हिंदी में उत्तर दीजिए।"},
                    {"role": "user", "content": prompt}
                ],
                return_tensors="pt",
                add_generation_prompt=True
            ).to(device)

            max_tokens = 100
            generated = input_ids
            past_key_values = None
            token_count = 0
            first_token_time = None

            model.eval()
            start = time.time()

            for _ in range(max_tokens):
                outputs = model(input_ids=generated[:, -1:], past_key_values=past_key_values, use_cache=True)
                logits = outputs.logits[:, -1, :]
                past_key_values = outputs.past_key_values

                next_token = torch.argmax(logits, dim=-1)
                generated = torch.cat([generated, next_token.unsqueeze(-1)], dim=-1)

                token_count += 1
                if first_token_time is None:
                    first_token_time = time.time()

                if next_token.item() == tokenizer.eos_token_id:
                    break

            end = time.time()
            ttft_ms = (first_token_time - start) * 1000 if first_token_time else -1
            tps = token_count / (end - first_token_time) if first_token_time else 0

            print(f"Prompt {idx} Trial {trial} -> TTFT: {ttft_ms:.2f} ms | TPS: {tps:.2f}")
            writer.writerow([idx, prompt, trial, ttft_ms, tps])

print(f"\n✅ Done. Results saved to: {csv_path}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Prompt 1 Trial 1 -> TTFT: 149.23 ms | TPS: 66.29
Prompt 1 Trial 2 -> TTFT: 97.55 ms | TPS: 62.63
Prompt 1 Trial 3 -> TTFT: 96.79 ms | TPS: 64.44
Prompt 2 Trial 1 -> TTFT: 93.62 ms | TPS: 63.70
Prompt 2 Trial 2 -> TTFT: 93.46 ms | TPS: 60.81
Prompt 2 Trial 3 -> TTFT: 92.83 ms | TPS: 64.03
Prompt 3 Trial 1 -> TTFT: 93.82 ms | TPS: 64.06
Prompt 3 Trial 2 -> TTFT: 94.11 ms | TPS: 61.94
Prompt 3 Trial 3 -> TTFT: 93.87 ms | TPS: 65.08
Prompt 4 Trial 1 -> TTFT: 94.49 ms | TPS: 61.06
Prompt 4 Trial 2 -> TTFT: 142.50 ms | TPS: 65.85
Prompt 4 Trial 3 -> TTFT: 93.70 ms | TPS: 64.75
Prompt 5 Trial 1 -> TTFT: 143.93 ms | TPS: 64.11
Prompt 5 Trial 2 -> TTFT: 95.42 ms | TPS: 65.87
Prompt 5 Trial 3 -> TTFT: 147.66 ms | TPS: 64.28
Prompt 6 Trial 1 -> TTFT: 99.97 ms | TPS: 65.78
Prompt 6 Trial 2 -> TTFT: 147.55 ms | TPS: 67.87
Prompt 6 Trial 3 -> TTFT: 97.77 ms | TPS: 61.49
Prompt 7 Trial 1 -> TTFT: 145.09 ms | TPS: 66.22
Prompt 7 Trial 2 -> TTFT: 96.74 ms | TPS: 61.77
Prompt 7 Trial 3 -> TTFT: 141.70 m

In [3]:
#cloud
import csv
import time
import openai

# === OpenAI-Compatible Client Setup ===
client = openai.OpenAI(
    base_url="https://api.sambanova.ai/v1",
    api_key="4a81daa9-5f3d-409b-9f30-ebedb379219a"
)

# === Prompts ===
prompts = [
    "भारत का प्रधानमंत्री कौन है?",
    "ताजमहल कहाँ स्थित है?",
    "पानी क्यों जरूरी होता है?",
    "भारत में कितने राज्य हैं?",
    "स्वास्थ्य ही धन क्यों है?",
    "गर्मी से कैसे बचा जा सकता है?",
    "आपका नाम क्या है?",
    "कृपया मुझे मौसम के बारे में बताइए।",
    "कंप्यूटर कैसे काम करता है?",
    "पृथ्वी सूरज के चारों ओर क्यों घूमती है?"
]

# === CSV Setup ===
csv_path = "cloud_llm_streaming_latency.csv"
with open(csv_path, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Prompt_ID", "Prompt", "Trial", "TTFT_ms", "TPS"])

    for idx, prompt in enumerate(prompts, 1):
        for trial in range(1, 4):
            try:
                token_count = 0
                first_token_time = None
                start_time = time.time()

                stream = client.chat.completions.create(
                    model="Meta-Llama-3.2-3B-Instruct",  # Change if needed
                    messages=[
                        {"role": "system", "content": "आप एक मददगार AI सहायक हैं। हिंदी में उत्तर दीजिए।"},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.7,
                    max_tokens=256,
                    stream=True
                )

                for chunk in stream:
                    if hasattr(chunk.choices[0].delta, "content"):
                        token_text = chunk.choices[0].delta.content
                        if token_text.strip():  # Count only non-empty tokens
                            token_count += 1
                            if first_token_time is None:
                                first_token_time = time.time()
                            print(token_text, end="", flush=True)

                end_time = time.time()

                ttft = (first_token_time - start_time) * 1000 if first_token_time else -1
                tps = token_count / (end_time - first_token_time) if first_token_time else 0

                print(f"\nTTFT: {ttft:.2f} ms | TPS: {tps:.2f} tokens/sec\n")
                writer.writerow([idx, prompt, trial, ttft, tps])

            except Exception as e:
                print(f"❌ Error for prompt {idx}, trial {trial}: {e}")
                writer.writerow([idx, prompt, trial, -1, 0])


भारत का वर्तमान प्रधानमंत्री श्री नरेंद्र मोदी हैं।
TTFT: 1208.27 ms | TPS: 110.86 tokens/sec

भारत का वर्तमान प्रधानमंत्री श्री नरेंद्र मोदी हैं।
TTFT: 702.92 ms | TPS: 71.34 tokens/sec

भारत का वर्तमान प्रधानमंत्री श्री नरेंद्र मोदी हैं।
TTFT: 660.61 ms | TPS: 108.45 tokens/sec

ताजमहल भारत के उत्तर प्रदेश राज्य में स्थित है, विशेष रूप से आगरा शहर में। यह एक प्रसिद्ध मकबरा है जो मुगल सम्राट शाहजहाँ द्वारा अपनी पत्नी मुमताज महल की याद में बनवाया गया था। ताजमहल का निर्माण 1632 से 1653 तक चला, और यह दुनिया के सात अजूबों में से एक है।
TTFT: 656.69 ms | TPS: 86.78 tokens/sec

ताजमहल भारत के उत्तर प्रदेश राज्य में स्थित है, विशेष रूप से आगरा शहर में। यह एक प्रसिद्ध मकबरा है जो मुगल सम्राट शाहजहाँ द्वारा अपनी पत्नी मुमताज महल की याद में बनवाया गया था। ताजमहल का निर्माण 1632 से 1653 तक चला, और यह दुनिया के सात अजूबों में से एक है।
TTFT: 654.32 ms | TPS: 86.87 tokens/sec

ताजमहल भारत के उत्तर प्रदेश राज्य में स्थित है, विशेष रूप से आगरा शहर में। यह एक प्रसिद्ध मकबरा है जो मुगल सम्राट शाहजहाँ 

In [1]:
#local
import csv
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup
model_id = "meta-llama/Llama-3.2-1B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    trust_remote_code=True
)

# Hindi prompts
prompts = [
    "भारत का प्रधानमंत्री कौन है?",
    "ताजमहल कहाँ स्थित है?",
    "पानी क्यों जरूरी होता है?",
    "भारत में कितने राज्य हैं?",
    "स्वास्थ्य ही धन क्यों है?",
    "गर्मी से कैसे बचा जा सकता है?",
    "आपका नाम क्या है?",
    "कृपया मुझे मौसम के बारे में बताइए।",
    "कंप्यूटर कैसे काम करता है?",
    "पृथ्वी सूरज के चारों ओर क्यों घूमती है?"
]

# CSV output
csv_path = "llm_streaming_llama_1B_latency.csv"
with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Prompt_ID", "Prompt", "Trial", "TTFT_ms", "TPS"])

    for idx, prompt in enumerate(prompts, 1):
        for trial in range(1, 4):
            input_ids = tokenizer.apply_chat_template(
                [
                    {"role": "system", "content": "आप एक मददगार AI सहायक हैं। हिंदी में उत्तर दीजिए।"},
                    {"role": "user", "content": prompt}
                ],
                return_tensors="pt",
                add_generation_prompt=True
            ).to(device)

            max_tokens = 100
            generated = input_ids
            past_key_values = None
            token_count = 0
            first_token_time = None

            model.eval()
            start = time.time()

            for _ in range(max_tokens):
                outputs = model(input_ids=generated[:, -1:], past_key_values=past_key_values, use_cache=True)
                logits = outputs.logits[:, -1, :]
                past_key_values = outputs.past_key_values

                next_token = torch.argmax(logits, dim=-1)
                generated = torch.cat([generated, next_token.unsqueeze(-1)], dim=-1)

                token_count += 1
                if first_token_time is None:
                    first_token_time = time.time()

                if next_token.item() == tokenizer.eos_token_id:
                    break

            end = time.time()
            ttft_ms = (first_token_time - start) * 1000 if first_token_time else -1
            tps = token_count / (end - first_token_time) if first_token_time else 0

            print(f"Prompt {idx} Trial {trial} -> TTFT: {ttft_ms:.2f} ms | TPS: {tps:.2f}")
            writer.writerow([idx, prompt, trial, ttft_ms, tps])

print(f"\n✅ Done. Results saved to: {csv_path}")


Prompt 1 Trial 1 -> TTFT: 139.15 ms | TPS: 140.37
Prompt 1 Trial 2 -> TTFT: 52.06 ms | TPS: 145.99
Prompt 1 Trial 3 -> TTFT: 52.87 ms | TPS: 150.26
Prompt 2 Trial 1 -> TTFT: 52.11 ms | TPS: 132.95
Prompt 2 Trial 2 -> TTFT: 51.74 ms | TPS: 150.61
Prompt 2 Trial 3 -> TTFT: 51.99 ms | TPS: 151.29
Prompt 3 Trial 1 -> TTFT: 51.63 ms | TPS: 151.78
Prompt 3 Trial 2 -> TTFT: 52.39 ms | TPS: 148.17
Prompt 3 Trial 3 -> TTFT: 52.14 ms | TPS: 137.12
Prompt 4 Trial 1 -> TTFT: 75.22 ms | TPS: 156.29
Prompt 4 Trial 2 -> TTFT: 52.92 ms | TPS: 147.75
Prompt 4 Trial 3 -> TTFT: 53.70 ms | TPS: 151.16
Prompt 5 Trial 1 -> TTFT: 53.58 ms | TPS: 150.05
Prompt 5 Trial 2 -> TTFT: 53.96 ms | TPS: 136.65
Prompt 5 Trial 3 -> TTFT: 54.21 ms | TPS: 150.76
Prompt 6 Trial 1 -> TTFT: 53.85 ms | TPS: 150.56
Prompt 6 Trial 2 -> TTFT: 77.61 ms | TPS: 156.24
Prompt 6 Trial 3 -> TTFT: 52.41 ms | TPS: 149.47
Prompt 7 Trial 1 -> TTFT: 52.46 ms | TPS: 135.85
Prompt 7 Trial 2 -> TTFT: 52.75 ms | TPS: 151.01
Prompt 7 Trial 3 ->

In [3]:
#cloud
import csv
import time
import openai

# === OpenAI-Compatible Client Setup ===
client = openai.OpenAI(
    base_url="https://api.sambanova.ai/v1",
    api_key="4a81daa9-5f3d-409b-9f30-ebedb379219a"
)

# === Prompts ===
prompts = [
    "भारत का प्रधानमंत्री कौन है?",
    "ताजमहल कहाँ स्थित है?",
    "पानी क्यों जरूरी होता है?",
    "भारत में कितने राज्य हैं?",
    "स्वास्थ्य ही धन क्यों है?",
    "गर्मी से कैसे बचा जा सकता है?",
    "आपका नाम क्या है?",
    "कृपया मुझे मौसम के बारे में बताइए।",
    "कंप्यूटर कैसे काम करता है?",
    "पृथ्वी सूरज के चारों ओर क्यों घूमती है?"
]

# === CSV Setup ===
csv_path = "cloud_llm_streaming_latency.csv"
with open(csv_path, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Prompt_ID", "Prompt", "Trial", "TTFT_ms", "TPS"])

    for idx, prompt in enumerate(prompts, 1):
        for trial in range(1, 4):
            try:
                token_count = 0
                first_token_time = None
                start_time = time.time()

                stream = client.chat.completions.create(
                    model="Meta-Llama-3.2-1B-Instruct",  # Change if needed
                    messages=[
                        {"role": "system", "content": "आप एक मददगार AI सहायक हैं। हिंदी में उत्तर दीजिए।"},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.7,
                    max_tokens=256,
                    stream=True
                )

                for chunk in stream:
                    if hasattr(chunk.choices[0].delta, "content"):
                        token_text = chunk.choices[0].delta.content
                        if token_text.strip():  # Count only non-empty tokens
                            token_count += 1
                            if first_token_time is None:
                                first_token_time = time.time()
                            print(token_text, end="", flush=True)

                end_time = time.time()

                ttft = (first_token_time - start_time) * 1000 if first_token_time else -1
                tps = token_count / (end_time - first_token_time) if first_token_time else 0

                print(f"\nTTFT: {ttft:.2f} ms | TPS: {tps:.2f} tokens/sec\n")
                writer.writerow([idx, prompt, trial, ttft, tps])

            except Exception as e:
                print(f"❌ Error for prompt {idx}, trial {trial}: {e}")
                writer.writerow([idx, prompt, trial, -1, 0])


भारत का वर्तमान प्रधानमंत्री नरेंद्र मोदी हैं। वह भारत के 14वें प्रधानमंत्री हैं और 2014 से 2023 तक के लिए कार्यरत हैं।
TTFT: 1240.11 ms | TPS: 263.42 tokens/sec

भारत का वर्तमान प्रधानमंत्री नरेंद्र मोदी हैं। वह भारत के 14वें प्रधानमंत्री हैं और 2014 से 2023 तक के लिए कार्यरत हैं।
TTFT: 637.98 ms | TPS: 131.40 tokens/sec

भारत का वर्तमान प्रधानमंत्री नरेंद्र मोदी हैं। वह भारत के 14वें प्रधानमंत्री हैं और 2014 से 2023 तक के लिए कार्यरत हैं।
TTFT: 646.49 ms | TPS: 164.57 tokens/sec

ताजमहल भारत के उत्तर प्रदेश राज्य के सिलहा शहर में स्थित एक ऐतिहासिक और सांस्कृतिक स्थल है। यह महल 1632 में शाहजहाँ और मुमताज महल द्वारा बनाया गया था और 1923 में बाद में संपत्ति को संग्रहालय के रूप में संरक्षित किया गया था।
TTFT: 707.12 ms | TPS: 93.14 tokens/sec

ताजमहल भारत के उत्तर प्रदेश राज्य के सिलहा शहर में स्थित एक ऐतिहासिक और सांस्कृतिक स्थल है। यह महल 1632 में शाहजहाँ और मुमताज महल द्वारा बनाया गया था और 1923 में बाद में संपत्ति को संग्रहालय के रूप में संरक्षित किया गया था।
TTFT: 640.99 ms | TPS: 1

In [4]:
#local
import csv
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup
model_id = "google/gemma-2-2b-it"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    trust_remote_code=True
)

# Hindi prompts
prompts = [
    "भारत का प्रधानमंत्री कौन है?",
    "ताजमहल कहाँ स्थित है?",
    "पानी क्यों जरूरी होता है?",
    "भारत में कितने राज्य हैं?",
    "स्वास्थ्य ही धन क्यों है?",
    "गर्मी से कैसे बचा जा सकता है?",
    "आपका नाम क्या है?",
    "कृपया मुझे मौसम के बारे में बताइए।",
    "कंप्यूटर कैसे काम करता है?",
    "पृथ्वी सूरज के चारों ओर क्यों घूमती है?"
]

# CSV output
csv_path = "llm_streaming_gemma_2B_latency.csv"
with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Prompt_ID", "Prompt", "Trial", "TTFT_ms", "TPS"])

    for idx, prompt in enumerate(prompts, 1):
        for trial in range(1, 4):
            input_ids = tokenizer.apply_chat_template(
                [
                    {"role": "system", "content": "आप एक मददगार AI सहायक हैं। हिंदी में उत्तर दीजिए।"},
                    {"role": "user", "content": prompt}
                ],
                return_tensors="pt",
                add_generation_prompt=True
            ).to(device)

            max_tokens = 100
            generated = input_ids
            past_key_values = None
            token_count = 0
            first_token_time = None

            model.eval()
            start = time.time()

            for _ in range(max_tokens):
                outputs = model(input_ids=generated[:, -1:], past_key_values=past_key_values, use_cache=True)
                logits = outputs.logits[:, -1, :]
                past_key_values = outputs.past_key_values

                next_token = torch.argmax(logits, dim=-1)
                generated = torch.cat([generated, next_token.unsqueeze(-1)], dim=-1)

                token_count += 1
                if first_token_time is None:
                    first_token_time = time.time()

                if next_token.item() == tokenizer.eos_token_id:
                    break

            end = time.time()
            ttft_ms = (first_token_time - start) * 1000 if first_token_time else -1
            tps = token_count / (end - first_token_time) if first_token_time else 0

            print(f"Prompt {idx} Trial {trial} -> TTFT: {ttft_ms:.2f} ms | TPS: {tps:.2f}")
            writer.writerow([idx, prompt, trial, ttft_ms, tps])

print(f"\n✅ Done. Results saved to: {csv_path}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

TemplateError: System role not supported